In [1]:
from matplotlib import rcParams
import matplotlib.pyplot as plt
%matplotlib inline
rcParams['figure.figsize'] = (13, 6)
plt.style.use('ggplot')
from nilmtk import DataSet
redd = DataSet('data/redd.h5')
import pickle
import numpy as np
import pandas as pd

OSError: No such file as data/redd.h5

# 1. Extracting long continuous data

- app_time_list: Time stamp data in appliances.
- main_time_list: Time stamp data in main.
- index: longest continuous data range index as appliances.

# 1.1. Time extracting from main and app

you can easily load app_time_list and main_time_list from pickle

In [2]:
import pickle

with open ('redd_app_time', 'rb') as fp:
    app_time_list = pickle.load(fp)
    
with open ('redd_main_time', 'rb') as fp:
    main_time_list = pickle.load(fp)
    
with open('redd_diffreq_time', 'rb') as fp:
    diffreq_app_time_list = pickle.load(fp)

Now you have 
- app_time_list
- main_time_list
- diffreq_app_time_list

      diffreq: [[1, 'washer dryer'], [3, 'washer dryer'], [5, 'washer dryer'], [5, 'electric space heater']]

In [3]:
p = 20
result_list = [0,0,0,0,0,0]
for i in [1, 3, 5]:
    main = redd.buildings[i+1].elec.mains().load().next()
    length = len(app_time_list[i])
    count = 0
    result = []
    tm = main_time_list[i][0]
    for j in range(p, length-1):
        time = app_time_list[i][j+1]
        #time diff for index j & nan test for main
        if time - app_time_list[i][j] <= p and not np.isnan(main.values[time-tm]):
            count += 1
        else:
            result.append([count, j])
            count = 0
    result.append([count, j])            
    result_list[i] = result

for i in [0,2,4]:
    main = redd.buildings[i+1].elec.mains().load().next()
    wash = redd.buildings[i+1].elec['washer dryer'].load().next()
    length = len(app_time_list[i])
    count = 0
    result = []
    tm = main_time_list[i][0]
    ta = diffreq_app_time_list[i/2][0]
    for j in range(p, length-20):
        time = app_time_list[i][j+1]
        time_wash = (time - ta) / 3
        #time diff for index j & nan test for main
        if time - app_time_list[i][j] <= p and not np.isnan(main.values[time-tm]) and not np.isnan(wash.values[time_wash]):
             count += 1
        else:
            result.append([count, j])
            count = 0
    result.append([count, j])            
    result_list[i] = result
    
    
ind_list = []
for i in range(6):
    ind = []
    for res in result_list[i]:
        if res[0] > 15000:
            ind.append(res)
    ind_list.append(ind)
    

Loading data for meter ElecMeterID(instance=1, building=2, dataset='REDD')     

nilmtk/utils.py:336: FutureWarning: 
.resample() is now a deferred operation
You called index(...) on this deferred object which materialized it into a series
by implicitly taking the mean.  Use .resample(...).mean() instead
  return resampled.index[0]
nilmtk/utils.py:390: FutureWarning: fill_method is deprecated to .resample()
the new syntax is .resample(...).ffill(limit=30)
  data = data.resample(**resample_kwargs)


Loading data for meter ElecMeterID(instance=2, building=2, dataset='REDD')     
Done loading data all meters for this chunk.
Loading data for meter ElecMeterID(instance=2, building=4, dataset='REDD')     
Done loading data all meters for this chunk.
Loading data for meter ElecMeterID(instance=2, building=6, dataset='REDD')     
Done loading data all meters for this chunk.
Loading data for meter ElecMeterID(instance=2, building=1, dataset='REDD')     
Done loading data all meters for this chunk.
Loading data for meter ElecMeterID(instance=10, building=1, dataset='REDD')     

nilmtk/utils.py:390: FutureWarning: fill_method is deprecated to .resample()
the new syntax is .resample(...).ffill(limit=17)
  data = data.resample(**resample_kwargs)


Loading data for meter ElecMeterID(instance=20, building=1, dataset='REDD')     
Done loading data all meters for this chunk.
Loading data for meter ElecMeterID(instance=2, building=3, dataset='REDD')     
Done loading data all meters for this chunk.
Loading data for meter ElecMeterID(instance=14, building=3, dataset='REDD')     
Done loading data all meters for this chunk.
Loading data for meter ElecMeterID(instance=2, building=5, dataset='REDD')     
Done loading data all meters for this chunk.
Loading data for meter ElecMeterID(instance=9, building=5, dataset='REDD')     
Done loading data all meters for this chunk.


In [4]:
ind_list = []
for i in range(6):
    for res in result_list[i]:
        if res[0] > 20000:
            ind_list.append([i, res[0], res[1]])
    

In [5]:
ind_list

[[0, 20917, 25258],
 [0, 23184, 60439],
 [0, 25257, 98950],
 [0, 27078, 126029],
 [0, 50520, 192424],
 [0, 24840, 217691],
 [0, 32540, 288469],
 [0, 29059, 317529],
 [0, 22848, 340386],
 [0, 30326, 475717],
 [0, 26218, 745857],
 [1, 31028, 33150],
 [1, 43382, 82757],
 [1, 26456, 152497],
 [1, 21643, 174142],
 [1, 30298, 204449],
 [1, 89455, 293915],
 [1, 20976, 314905],
 [2, 31921, 88671],
 [2, 21400, 114360],
 [2, 26251, 140612],
 [2, 40738, 201317],
 [2, 26546, 332921],
 [2, 70860, 403799],
 [3, 33028, 113951],
 [3, 20295, 167285],
 [3, 38755, 237101],
 [3, 20471, 260108],
 [3, 36745, 296854],
 [3, 97861, 440716],
 [4, 21669, 80396],
 [5, 153842, 182940],
 [5, 40125, 267614],
 [5, 73400, 341015],
 [5, 35951, 376966]]

# 2. Missing value filling

In [6]:
data_keys =[
    ['washer dryer', 'dish washer', 'fridge', 'microwave', 'electric space heater', 'electric stove'],
    ['washer dryer', 'dish washer', 'fridge', 'microwave', 'waste disposal unit','electric stove'],
    ['washer dryer', 'dish washer', 'fridge', 'microwave', 'electric furnace', 'waste disposal unit', 'CE appliance'],
    ['washer dryer', 'dish washer', 'electric furnace', 'electric stove'],
    ['washer dryer', 'dish washer', 'fridge', 'microwave', 'electric space heater', 'electric furnace', 'waste disposal unit', 'CE appliance'],
    ['washer dryer', 'dish washer', 'fridge', 'electric space heater', 'electric stove', 'CE appliance']
]
for i in range(6):
    data_keys[i].sort()

In [7]:
house_iter = 0
house_before = 0

for index in ind_list:
    validation = []
    
    new_data_list = [[]]
    house = index[0]
    count = 0
    index_end = index[2]
    index_start = index_end - index[1] + 1
    #l: num_app
    l = 1+len(data_keys[house])
    print 'building' + str(house+1) + ', iter' + str(house_iter)

    #data load in df list
    df_list = []
    elec = redd.buildings[house+1].elec
    main = elec.mains().load().next()
    for key in data_keys[house]:
        df_list.append(elec[key].load().next())
        new_data_list.append([])
        
        
    #j means app no.
    for j in range(l-1):
        if (house in [0,2,4] and data_keys[house][j] == 'washer dryer') or (house==4 and data_keys[house][j]=='electric space heater'):
            app_index = app_time_list[house][index_start] - diffreq_app_time_list[house/2][0]
            new_data_list[j].append(df_list[j].values[app_index/3])
            
            validation.append(df_list[j].index[app_index/3].ctime())
        else:
            new_data_list[j].append(df_list[j].values[index_start])
            
            validation.append(df_list[j].index[index_start].ctime())
    main_index = app_time_list[house][index_start] - main_time_list[house][0]
    new_data_list[l-1].append(main.values[main_index])
    
    validation.append(main.index[main_index].ctime())
    print validation
    
    # j means app index.
    for j in range(index_start+1, index_end+1):
        if app_time_list[house][j] - app_time_list[house][j-1] <= 5:
            repeat = 1
        else:
            diff = app_time_list[house][j] - app_time_list[house][j-1]
            repeat = diff/3

        for o in xrange(repeat):
            for k in range(l-1):
                if (house in [0,2,4] and data_keys[house][k] == 'washer dryer') or (house==4 and data_keys[house][k]=='electric space heater'):
                    app_index = app_time_list[house][j] - diffreq_app_time_list[house/2][0]
                    new_data_list[k].append(df_list[k].values[app_index/3])
                else:
                    new_data_list[k].append(df_list[k].values[j])

            main_index = app_time_list[house][j] - main_time_list[house][0]
            new_data_list[l-1].append(main.values[main_index])
            
    print j
    
    columns = []
    for key in data_keys[house]:
        columns.append(key)
    columns.append('main')
    darr = np.array(new_data_list)
    n,x,_ = darr.shape
    
    if house_before != house:
        house_before = house
        house_iter = 0
        
    df = pd.DataFrame(columns=columns, data=darr.reshape(n,x).T)
    df.to_csv('REDD/redd_house'+str(house+1)+'_'+str(house_iter)+'.csv')
    house_iter += 1
    

building1, iter0
Loading data for meter ElecMeterID(instance=2, building=1, dataset='REDD')     
Done loading data all meters for this chunk.
Loading data for meter ElecMeterID(instance=20, building=1, dataset='REDD')     
Done loading data all meters for this chunk.
['Mon Apr 18 14:03:30 2011', 'Mon Apr 18 14:03:30 2011', 'Mon Apr 18 14:03:30 2011', 'Mon Apr 18 14:03:30 2011', 'Mon Apr 18 14:03:30 2011', 'Mon Apr 18 14:03:30 2011', 'Mon Apr 18 14:03:30 2011']
25258
building1, iter1
Loading data for meter ElecMeterID(instance=2, building=1, dataset='REDD')     
Done loading data all meters for this chunk.
Loading data for meter ElecMeterID(instance=20, building=1, dataset='REDD')     
Done loading data all meters for this chunk.
['Wed Apr 20 02:29:00 2011', 'Wed Apr 20 02:29:00 2011', 'Wed Apr 20 02:29:00 2011', 'Wed Apr 20 02:29:00 2011', 'Wed Apr 20 02:29:00 2011', 'Wed Apr 20 02:29:00 2011', 'Wed Apr 20 02:29:00 2011']
60439
building1, iter2
Loading data for meter ElecMeterID(instan